In [2]:
# Always needs to be done in Rasberry Pi
import findspark
findspark.init('/home/baxman/spark-2.4.7-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp_spam_detection').getOrCreate()

In [4]:
# Set data
data = spark.read.csv('/home/baxman/Codes/PySpark/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Natural_Language_Processing/smsspamcollection/SMSSpamCollection', inferSchema = True, sep = '\t')

In [5]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [6]:
# Relabeling
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1','text')

In [7]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [10]:
# Clean data
from pyspark.sql.functions import length
data = data.withColumn('length',length(data['text']))

In [11]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [12]:
# Investigating text legnths for ham and spam
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [33]:
# So spams are twice the size of hams
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,StringIndexer,IDF

In [40]:
# Define classes
tokenizer = Tokenizer(inputCol = 'text', outputCol = 'token_text')
stop_remove = StopWordsRemover(inputCol = 'token_text', outputCol = 'stop_token')
count_vec = CountVectorizer(inputCol = 'stop_token', outputCol = 'count_vec')
idf = IDF(inputCol = 'count_vec', outputCol = 'tf-idf')
ham_spam_to_num = StringIndexer(inputCol = 'class',outputCol='label')

In [41]:
# Clean data for ingestion
from pyspark.ml.feature import VectorAssembler
clean_up = VectorAssembler(inputCols = ['tf-idf', 'length'], outputCol = 'features')

In [42]:
# Build model - Naive Bayes
from pyspark.ml.classification import NaiveBayes
model = NaiveBayes()

In [43]:
# Using a pipeline
from pyspark.ml import Pipeline

In [44]:
# Define pipeline
data_pipe = Pipeline(stages = [ham_spam_to_num,tokenizer,
                              stop_remove,count_vec,idf,
                              clean_up])

In [45]:
# Fit pipeline functions
cleaner = data_pipe.fit(data)

In [46]:
clean_data = cleaner.transform(data)

In [47]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'count_vec',
 'tf-idf',
 'features']

In [48]:
# Choose only labels and features
clean_data = clean_data.select(['label','features'])

In [49]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [50]:
training,test = clean_data.randomSplit([0.7,0.3])

In [51]:
spam_detector = model.fit(training)

In [52]:
# Transform test data
test_results = spam_detector.transform(test)

In [53]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,13,...|[-613.93931082650...|[0.99999999999994...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-1000.9677029283...|[1.0,2.0863439930...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-801.40468332941...|[1.0,1.8301204152...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-876.25446133064...|[1.0,2.3855770430...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-886.00995293641...|[1.0,8.8686814069...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1365.0939170254...|[1.0,1.5897981194...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-665.38332905280...|[1.0,3.7396266164...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-830.59822762088...|[1.0,5.7308515153...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-756.43433174550...|[1.0,1.1887771067...|       0.0|
|  0.0|(13424,[0

In [54]:
# Evaluate
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [55]:
acc_eval = MulticlassClassificationEvaluator()

In [57]:
acc = acc_eval.evaluate(test_results)

In [60]:
print('ACCURACY IS:',acc)

ACCURACY IS: 0.9220774122416092


In [61]:
# 92% accuracy